# Week 5 Batch Processing

### Question 1: 

**Install Spark and PySpark** 

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?
- **Answer: 3.3.2**
- 2.1.4
- 1.2.3
- 5.4

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 22:39:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

### Question 2: 

**HVFHW June 2021**

Read it with Spark using the same schema as we did in the lessons.</br> 
We will use this dataset for all the remaining questions.</br>
Repartition it to 12 partitions and save it to parquet.</br>
What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.</br>


- 2MB
- **Answer: 24MB**
- 100MB
- 250MB

In [4]:
from pyspark.sql import types

In [5]:
# Set schema
schema = types.StructType([
    types.StructField("dispatching_base_num", types.StringType(), True),
    types.StructField("pickup_datetime", types.TimestampType(), True),
    types.StructField("dropoff_datetime", types.TimestampType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("SR_Flag", types.StringType(), True),
    types.StructField("Affiliated_base_number", types.StringType(), True)
])

In [6]:
# read with spark using schema
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv("fhvhv_tripdata_2021-06.csv.gz")

# Repartition to 12 partitions and save to parquet
df.repartition(12) \
    .write.parquet("data/", mode="overwrite")

In [7]:
!ls -lh data

total 591360
-rw-r--r--@ 1 koda  staff     0B Mar  6 22:40 _SUCCESS
-rw-r--r--@ 1 koda  staff    24M Mar  6 22:40 part-00000-ddb5cb5e-1906-4d7d-9cc0-bdac130506a6-c000.snappy.parquet
-rw-r--r--@ 1 koda  staff    24M Mar  6 22:40 part-00001-ddb5cb5e-1906-4d7d-9cc0-bdac130506a6-c000.snappy.parquet
-rw-r--r--@ 1 koda  staff    24M Mar  6 22:40 part-00002-ddb5cb5e-1906-4d7d-9cc0-bdac130506a6-c000.snappy.parquet
-rw-r--r--@ 1 koda  staff    24M Mar  6 22:40 part-00003-ddb5cb5e-1906-4d7d-9cc0-bdac130506a6-c000.snappy.parquet
-rw-r--r--@ 1 koda  staff    24M Mar  6 22:40 part-00004-ddb5cb5e-1906-4d7d-9cc0-bdac130506a6-c000.snappy.parquet
-rw-r--r--@ 1 koda  staff    24M Mar  6 22:40 part-00005-ddb5cb5e-1906-4d7d-9cc0-bdac130506a6-c000.snappy.parquet
-rw-r--r--@ 1 koda  staff    24M Mar  6 22:40 part-00006-ddb5cb5e-1906-4d7d-9cc0-bdac130506a6-c000.snappy.parquet
-rw-r--r--@ 1 koda  staff    24M Mar  6 22:40 part-00007-ddb5cb5e-1906-4d7d-9cc0-bdac130506a6-c000.snappy.parquet
-rw-r--r--@ 1 koda  

### Question 3: 

**Count records**  

How many taxi trips were there on June 15?</br></br>
Consider only trips that started on June 15.</br>

- 308,164
- 12,856
- **Answer: 452,470**
- 50,982

In [8]:
df.createOrReplaceTempView('fhvhv_data')

In [9]:
spark.sql("""
SELECT
    COUNT(1)
FROM
    fhvhv_data
WHERE
    pickup_datetime >= '2021-06-15 00:00:00' AND
    pickup_datetime <= '2021-06-15 23:59:59'
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



### Question 4: 

**Longest trip for each day**  

Now calculate the duration for each trip.</br>
How long was the longest trip in Hours?</br>

- **Answer: 66.87 Hours**
- 243.44 Hours
- 7.68 Hours
- 3.32 Hours

In [10]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [13]:
spark.sql("""
SELECT
    (unix_timestamp(dropoff_datetime)-unix_timestamp(pickup_datetime)) / 3600 as trip_duration
FROM
    fhvhv_data
ORDER BY
    trip_duration DESC
LIMIT 5
""").show()

+------------------+
|     trip_duration|
+------------------+
|  66.8788888888889|
|25.549722222222222|
|19.980833333333333|
|18.197222222222223|
|16.466944444444444|
+------------------+



### Question 5: 

**User Interface**

 Spark’s User Interface which shows application's dashboard runs on which local port?</br>

- 80
- 443
- **Answer: 4040**
- 8080

### Question 6: 

**Most frequent pickup location zone**

Load the zone lookup data into a temp view in Spark</br>
[Zone Data](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv)</br>

Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?</br>

- East Chelsea
- Astoria
- Union Sq
- **Answer: Crown Heights North**

In [17]:
# read with spark
df_zones = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("taxi_zone_lookup.csv")

df_zones.createOrReplaceTempView('zones_data')

In [35]:
spark.sql("""
SELECT
    z.Zone, Count(*) as count
FROM
    fhvhv_data f,
    zones_data z
WHERE
    f.PULocationID = z.LocationID
GROUP BY
    z.Zone
ORDER BY
    count DESC
LIMIT 10
""").show()

+--------------------+------+
|                Zone| count|
+--------------------+------+
| Crown Heights North|231279|
|        East Village|221244|
|         JFK Airport|188867|
|      Bushwick South|187929|
|       East New York|186780|
|TriBeCa/Civic Center|164344|
|   LaGuardia Airport|161596|
|            Union Sq|158937|
|        West Village|154698|
|             Astoria|152493|
+--------------------+------+

